In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision
from torchvision import transforms,datasets
import os,PIL,pathlib

In [2]:
device=torch.device("cuda"if torch.cuda.is_available()else "cpu")
device

device(type='cuda')

In [3]:
data_dir="IMAGES/"
data_dir=pathlib.Path(data_dir)
data_paths=list(data_dir.glob("*"))
classeName=[str(path).split("\\")[1]for path in data_paths]
classeName

['carzing',
 'inclusion',
 'patches',
 'pittted_surface',
 'rolled_in_scale',
 'scratch']

In [4]:
total_datadir="IMAGES/"
train_transforms=transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(
    mean=[0.485,0.456,0.406],
    std=[0.229,0.224,0.225])
])
total_data=datasets.ImageFolder(total_datadir,transform=train_transforms)
total_data

Dataset ImageFolder
    Number of datapoints: 1800
    Root location: IMAGES/
    StandardTransform
Transform: Compose(
               Resize(size=[224, 224], interpolation=bilinear)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [5]:
train_size=int(0.8*len(total_data))
test_size=len(total_data)-train_size
train_dataset,test_dataset=torch.utils.data.random_split(total_data,[train_size,test_size])
train_dataset,test_dataset

(<torch.utils.data.dataset.Subset at 0x18d7b6c2438>,
 <torch.utils.data.dataset.Subset at 0x18d7b6c20f0>)

In [6]:
batch_size=32
train_dl=torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True,num_workers=1)
test_dl=torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=True,num_workers=1)

In [7]:
import torch.nn.functional as F

class Network_bn(nn.Module):
    def __init__(self):
        super(Network_bn, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(12)
        self.pool = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(12)
        self.conv4 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=0)
        self.bn4 = nn.BatchNorm2d(24)
        self.conv5 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=0)
        self.bn5 = nn.BatchNorm2d(24)
        self.fc1 = nn.Linear(24*49*49, len(classeName))

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x))) #220     
        x = F.relu(self.bn2(self.conv2(x)))  #216  
        x = F.relu(self.bn2(self.conv3(x))) #212
        x = self.pool(x)        #106                
        x = F.relu(self.bn4(self.conv4(x)))   #102  
        x = F.relu(self.bn5(self.conv5(x)))  #98
        x = self.pool(x)        #49                
        x = x.view(-1, 24*49*49)
        x = self.fc1(x)

        return x

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

model = Network_bn().to(device)
model

Using cuda device


Network_bn(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(12, 12, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(12, 12, kernel_size=(5, 5), stride=(1, 1))
  (bn3): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1))
  (bn4): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1))
  (bn5): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=57624, out_features=6, bias=True)
)

In [8]:
loss_fn=nn.CrossEntropyLoss()
learn_rate=1e-4
opt=torch.optim.Adam(model.parameters(),lr=learn_rate)

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)  # 训练集的大小，一共60000张图片
    num_batches = len(dataloader)   # 批次数目，1875（60000/32）

    train_loss, train_acc = 0, 0  # 初始化训练损失和正确率
    
    for X, y in dataloader:  # 获取图片及其标签
        X, y = X.to(device), y.to(device)
        
        # 计算预测误差
        pred = model(X)          # 网络输出
        loss = loss_fn(pred, y)  # 计算网络输出和真实值之间的差距，targets为真实值，计算二者差值即为损失
        
        # 反向传播
        optimizer.zero_grad()  # grad属性归零
        loss.backward()        # 反向传播
        optimizer.step()       # 每一步自动更新
        
        # 记录acc与loss
        train_acc  += (pred.argmax(1) == y).type(torch.float).sum().item()
        train_loss += loss.item()
            
    train_acc  /= size
    train_loss /= num_batches

    return train_acc, train_loss

In [10]:
def test (dataloader, model, loss_fn):
    size        = len(dataloader.dataset)  # 测试集的大小，一共10000张图片
    num_batches = len(dataloader)          # 批次数目，313（10000/32=312.5，向上取整）
    test_loss, test_acc = 0, 0
    
    # 当不进行训练时，停止梯度更新，节省计算内存消耗
    with torch.no_grad():
        for imgs, target in dataloader:
            imgs, target = imgs.to(device), target.to(device)
            
            # 计算loss
            target_pred = model(imgs)
            loss        = loss_fn(target_pred, target)
            
            test_loss += loss.item()
            test_acc  += (target_pred.argmax(1) == target).type(torch.float).sum().item()

    test_acc  /= size
    test_loss /= num_batches

    return test_acc, test_loss

In [11]:
epochs=20
train_loss=[]
train_acc=[]
test_loss=[]
test_acc=[]
for epoch in range(epochs):
    model.train()
    epoch_train_acc,epoch_train_loss=train(train_dl,model,loss_fn,opt)
    model.eval()
    epoch_test_acc,epoch_test_loss=test(test_dl,model,loss_fn)
    train_acc.append(epoch_train_acc)
    train_loss.append(epoch_train_loss)
    test_acc.append(epoch_test_acc)
    test_loss.append(epoch_test_loss)
    
    template = ('Epoch:{:2d}, Train_acc:{:.1f}%, Train_loss:{:.3f}, Test_acc:{:.1f}%，Test_loss:{:.3f}')
    print(template.format(epoch+1, epoch_train_acc*100, epoch_train_loss, epoch_test_acc*100, epoch_test_loss))
print('Done')

Epoch: 1, Train_acc:70.1%, Train_loss:0.785, Test_acc:15.6%，Test_loss:5.682
Epoch: 2, Train_acc:85.1%, Train_loss:0.439, Test_acc:35.0%，Test_loss:6.269
Epoch: 3, Train_acc:93.5%, Train_loss:0.195, Test_acc:32.8%，Test_loss:7.207
Epoch: 4, Train_acc:93.8%, Train_loss:0.178, Test_acc:32.2%，Test_loss:8.536
Epoch: 5, Train_acc:95.5%, Train_loss:0.156, Test_acc:29.7%，Test_loss:8.687
Epoch: 6, Train_acc:95.4%, Train_loss:0.130, Test_acc:29.7%，Test_loss:12.443
Epoch: 7, Train_acc:97.2%, Train_loss:0.094, Test_acc:30.8%，Test_loss:12.524
Epoch: 8, Train_acc:94.7%, Train_loss:0.159, Test_acc:31.1%，Test_loss:11.316
Epoch: 9, Train_acc:96.4%, Train_loss:0.114, Test_acc:29.7%，Test_loss:15.867
Epoch:10, Train_acc:98.7%, Train_loss:0.054, Test_acc:29.7%，Test_loss:14.691
Epoch:11, Train_acc:99.0%, Train_loss:0.042, Test_acc:30.8%，Test_loss:12.294
Epoch:12, Train_acc:99.0%, Train_loss:0.044, Test_acc:31.7%，Test_loss:14.164
Epoch:13, Train_acc:98.5%, Train_loss:0.044, Test_acc:31.9%，Test_loss:15.387
Epoc

In [12]:
from PIL import Image
classes=list(total_data.class_to_idx)
def pred_one_image(image_path,model,transform,classes):
    test_img=Image.open(image_path).convert("RGB")
    test_img=transform(test_img)
    img=test_img.to(device).unsqueeze(0)
    model.eval()
    output=model(img)
    _,pred=torch.max(output,1)
    pred_class=classes[pred]
    print("结果为：{}".format(pred_class))

In [13]:
path=r"F:\postgraduate\deeplearning365\8.pytorchweather\model.pt"
torch.save(model.state_dict(),path)
model.load_state_dict(torch.load(path,map_location=device))

<All keys matched successfully>

In [14]:
pred_one_image(image_path=r"F:\postgraduate\deeplearning365\8.pytorchweather\IMAGES\patches\patches_1.jpg",model=model,transform=train_transforms,classes=classes)


结果为：pittted_surface
